# Prezicerea satisfacției față de o companie aeriană

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("airline satisfaction").getOrCreate()

## Pregătirea datelor

In [ ]:
airline = spark.read.csv("/content/drive/MyDrive/Proiect Big Data/airline_satisfaction.csv", header=True, inferSchema=True)
airline_clean = airline.drop("_c0", "id")
airline_data = airline_clean.na.drop()
airline_data.show(5)

+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+
|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|
+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+---

Scopul următoarelor metode de Machine Learning este de a prezice nivelul general de satisfacție a unui pasager (**satisfied** sau **neutral or dissatisfied**) față de compania aeriană pe baza informațiilor despre zboruri și a evaluărilor diferitelor servicii.

Pentru toate modelele propuse vom transforma variabilele categoriale **Gender**, **Customer Type**, **Type of Travel** și **Class** în variabile numerice cu ajutorul *One-Hot Encoding*. De asemenea, valorile coloanei țintă **satisfaction** vor fi transformate în valori binare astfel:

* **satisfied** -> 1
* **neutral or dissatisfied** -> 0

Deoarece problema este una de clasificare binară, vom folosi pentru început **regresia logistică** pentru a face predicțiile.

Pentru a înlănțui pașii necesari pentru obținerea rezultatelor vom folosi un Pipeline.

In [ ]:
from pyspark.sql.functions import when, col

airline_data = airline_data.withColumn("label", when(col("satisfaction") == "satisfied", 1).otherwise(0))
airline_data.show(5)

+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+---------------+------------+----------------------+----------------+----------------+----------------+---------------+----------------+-----------+--------------------------+------------------------+--------------------+-----+
|Gender|    Customer Type|Age| Type of Travel|   Class|Flight Distance|Inflight wifi service|Departure/Arrival time convenient|Ease of Online booking|Gate location|Food and drink|Online boarding|Seat comfort|Inflight entertainment|On-board service|Leg room service|Baggage handling|Checkin service|Inflight service|Cleanliness|Departure Delay in Minutes|Arrival Delay in Minutes|        satisfaction|label|
+------+-----------------+---+---------------+--------+---------------+---------------------+---------------------------------+----------------------+-------------+--------------+-------

In [ ]:
# împărțirea în date de train și test
train_airline_data, test_airline_data = airline_data.randomSplit([0.7, 0.3], seed=22)

## Regresie Logistică

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression

# transformarea variabilelor categoriale
gender_indexer = StringIndexer(inputCol="Gender", outputCol="Gender Index")
customer_type_indexer = StringIndexer(inputCol="Customer Type", outputCol="Customer Type Index")
travel_type_indexer = StringIndexer(inputCol="Type of Travel", outputCol="Travel Type Index")
class_indexer = StringIndexer(inputCol="Class", outputCol="Class Index")
ohe = OneHotEncoder(inputCols=["Gender Index", "Customer Type Index", "Travel Type Index", "Class Index"], \
                    outputCols=["Gender OHE", "Customer Type OHE", "Travel Type OHE", "Class OHE"])

# alegerea și asamblarea coloanelor de tip feature
cols = [c for c in airline_data.columns if c not in ["Gender", "Customer Type", "Type of Travel", "Class", "satisfaction", "label"]]
cols.extend(["Gender OHE", "Customer Type OHE", "Travel Type OHE", "Class OHE"])
assembler = VectorAssembler(inputCols=cols, outputCol="features")

# definirea modelului
log_reg = LogisticRegression(featuresCol="features", labelCol="label")

In [ ]:
# definirea pipeline-ului
from pyspark.ml import Pipeline

log_reg_pipeline = Pipeline(stages=[gender_indexer, customer_type_indexer, travel_type_indexer, class_indexer, ohe, assembler, log_reg])

In [ ]:
# antrenarea modelului
fit_log_reg_model = log_reg_pipeline.fit(train_airline_data)

In [ ]:
# efectuarea predicțiilor
pred_log_reg = fit_log_reg_model.transform(test_airline_data)

In [ ]:
# evaluarea modelului cu ajutorul metricii Area Under the Curve
from pyspark.ml.evaluation import BinaryClassificationEvaluator

log_reg_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
log_reg_eval.evaluate(pred_log_reg)

0.8690452202942274

In [ ]:
# afișarea matricei de confuzie
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

preds_and_labels_log_reg = pred_log_reg.select(["prediction", "label"]).withColumn("label", col("label").cast(FloatType())).orderBy("prediction")
metrics = MulticlassMetrics(preds_and_labels_log_reg.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[16041.  1677.]
 [ 2255. 11227.]]


Fiind o problemă de clasificare, un al doilea model pe care îl putem încerca este **Random Forest**.

Deoarece acest algoritm are un număr semnificativ de hiperparametrii vom dori optimizarea acestora prin intermediul **Grid Search** și al metodei **Cross Validation** cu 3 diviziuni.

De asemenea, vom utiliza un pipeline pentru crearea caracteristicilor în formatul dorit și antrenarea modelului.

## Random Forest

In [ ]:
# instanțierea modelului
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=100)

In [ ]:
# definirea pipeline-ului
rf_pipeline = Pipeline(stages=[gender_indexer, customer_type_indexer, travel_type_indexer, class_indexer, ohe, assembler, rf])

In [ ]:
# definirea gridului de parametrii
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = ParamGridBuilder() \
            .addGrid(rf.bootstrap, [True]) \
            .addGrid(rf.maxDepth, [5, 10, 15]) \
            .addGrid(rf.numTrees, [100, 200]) \
            .build()

In [ ]:
# aplicarea Cross Validation cu 3 diviziuni
from pyspark.ml.tuning import CrossValidator

crossval = CrossValidator(estimator=rf_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)

In [ ]:
# antrenarea modelului
rf_fit = crossval.fit(train_airline_data)

In [ ]:
# efectuarea predicțiilor
pred_rf = rf_fit.transform(test_airline_data)

In [ ]:
# evaluarea modelului cu ajutorul metricii Area Under the Curve
rf_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
rf_eval.evaluate(pred_rf)

0.9563298279382983

Observăm că optimizarea hiperparametrilor modelului Random Forest a adus o îmbunătățire semnificativă față de regresia logistică cu parametrii default.

In [ ]:
# afișarea matricei de confuzie
preds_and_labels_rf = pred_rf.select(["prediction", "label"]).withColumn("label", col("label").cast(FloatType())).orderBy("prediction")
metrics = MulticlassMetrics(preds_and_labels_rf.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

[[17281.   437.]
 [  845. 12637.]]


In [ ]:
# afișarea de metrici suplimentare (accuracy, precision, recall, f1)
print(f"Accuracy: {metrics.accuracy}")
print(f"Precision for 'satisfied': {metrics.precision(1.0)} \t Precision for 'neutral or dissatisfied': {metrics.precision(0.0)}")
print(f"Recall for 'satisfied': {metrics.recall(1.0)} \t Recall for 'neutral or dissatisfied': {metrics.recall(0.0)}")
print(f"f1 score for 'satisfied': {metrics.fMeasure(1.0)} \t f1 score for 'neutral or dissatisfied': {metrics.fMeasure(0.0)}")

Accuracy: 0.9589102564102564
Precision for 'satisfied': 0.9665748814440875 	 Precision for 'neutral or dissatisfied': 0.9533818823789032
Recall for 'satisfied': 0.9373238391929981 	 Recall for 'neutral or dissatisfied': 0.9753358166835986
f1 score for 'satisfied': 0.9517246573279109 	 f1 score for 'neutral or dissatisfied': 0.9642339024662425


Observăm cum clasificatorul are o performanță ușor mai bună pentru clasa **neutral or dissatisfied**.